In [1]:
# !pip install git+https://github.com/MPI-IS/mesh.git

In [2]:
# !pip install chumpy

In [1]:
import os
import cv2
import numpy as np
import tensorflow as tf
# from psbody.mesh import Mesh
# from psbody.mesh.meshviewer import MeshViewers
from utils.landmarks import load_embedding, tf_get_model_lmks, tf_project_points
from tf_smpl.batch_smpl import SMPL
from tensorflow.contrib.opt import ScipyOptimizerInterface as scipy_pt

In [2]:
def fit_lmk2d(target_img, target_2d_lmks, model_fname, lmk_face_idx, lmk_b_coords, weights):
    '''
    Fit FLAME to 2D landmarks
    :param target_2d_lmks      target 2D landmarks provided as (num_lmks x 3) matrix
    :param model_fname         saved FLAME model
    :param lmk_face_idx        face indices of the landmark embedding in the FLAME topology
    :param lmk_b_coords        barycentric coordinates of the landmark embedding in the FLAME topology
                                (i.e. weighting of the three vertices for the trinagle, the landmark is embedded in
    :param weights             weights of the individual objective functions
    :param visualize           visualize fitting progress
    :return: a mesh with the fitting results
    '''

    tf_trans = tf.Variable(np.zeros((1,3)), name="trans", dtype=tf.float64, trainable=True)
    tf_rot = tf.Variable(np.zeros((1,3)), name="rot", dtype=tf.float64, trainable=True)
    tf_pose = tf.Variable(np.zeros((1,12)), name="pose", dtype=tf.float64, trainable=True)
    tf_shape = tf.Variable(np.zeros((1,300)), name="shape", dtype=tf.float64, trainable=True)
    tf_exp = tf.Variable(np.zeros((1,100)), name="expression", dtype=tf.float64, trainable=True)
    smpl = SMPL(model_fname)
    tf_model = tf.squeeze(smpl(tf_trans,
                               tf.concat((tf_shape, tf_exp), axis=-1),
                               tf.concat((tf_rot, tf_pose), axis=-1)))

    with tf.Session() as session:
        # session.run(tf.global_variables_initializer())

        # Mirror landmark y-coordinates
        target_2d_lmks[:,1] = target_img.shape[0]-target_2d_lmks[:,1]

        lmks_3d = tf_get_model_lmks(tf_model, smpl.f, lmk_face_idx, lmk_b_coords)

        s2d = np.mean(np.linalg.norm(target_2d_lmks-np.mean(target_2d_lmks, axis=0), axis=1))
        s3d = tf.reduce_mean(tf.sqrt(tf.reduce_sum(tf.square(lmks_3d-tf.reduce_mean(lmks_3d, axis=0))[:, :2], axis=1)))
        tf_scale = tf.Variable(s2d/s3d, dtype=lmks_3d.dtype)

        # trans = 0.5*np.array((target_img.shape[0], target_img.shape[1]))/tf_scale
        # trans = 0.5 * s3d * np.array((target_img.shape[0], target_img.shape[1])) / s2d
        lmks_proj_2d = tf_project_points(lmks_3d, tf_scale, np.zeros(2))

        factor = max(max(target_2d_lmks[:,0]) - min(target_2d_lmks[:,0]),max(target_2d_lmks[:,1]) - min(target_2d_lmks[:,1]))
        lmk_dist = weights['lmk']*tf.reduce_sum(tf.square(tf.subtract(lmks_proj_2d, target_2d_lmks))) / (factor ** 2)
        neck_pose_reg = weights['neck_pose']*tf.reduce_sum(tf.square(tf_pose[:,:3]))
        jaw_pose_reg = weights['jaw_pose']*tf.reduce_sum(tf.square(tf_pose[:,3:6]))
        eyeballs_pose_reg = weights['eyeballs_pose']*tf.reduce_sum(tf.square(tf_pose[:,6:]))
        shape_reg = weights['shape']*tf.reduce_sum(tf.square(tf_shape))
        exp_reg = weights['expr']*tf.reduce_sum(tf.square(tf_exp))

        session.run(tf.global_variables_initializer())

        def on_step(*_):
            # print(tf_exp.numpy())
            pass

        print('Optimize rigid transformation')
        vars = [tf_scale, tf_trans, tf_rot]
        loss = lmk_dist
        optimizer = scipy_pt(loss=loss, var_list=vars, method='L-BFGS-B', options={'disp': 1, 'ftol': 5e-6})
        optimizer.minimize(session, fetches=[tf_model, tf_scale, tf.constant(smpl.f), tf.constant(target_img), tf.constant(target_2d_lmks), lmks_proj_2d], loss_callback=on_step)

        print('Optimize model parameters')
        vars = [tf_scale, tf_trans[:2], tf_rot, tf_pose, tf_shape, tf_exp]
        loss = lmk_dist + shape_reg + exp_reg + neck_pose_reg + jaw_pose_reg + eyeballs_pose_reg
        optimizer = scipy_pt(loss=loss, var_list=vars, method='L-BFGS-B', options={'disp': 0, 'ftol': 1e-7})
        optimizer.minimize(session, fetches=[tf_model, tf_scale, tf.constant(smpl.f), tf.constant(target_img), tf.constant(target_2d_lmks), lmks_proj_2d,
                                             lmk_dist, shape_reg, exp_reg, neck_pose_reg, jaw_pose_reg, eyeballs_pose_reg], loss_callback=on_step)

        print('Fitting done')
        np_verts, np_scale, np_shape, np_exp, np_trans, np_rot, np_pose = session.run([tf_model, tf_scale, tf_shape, tf_exp, tf_trans, tf_rot, tf_pose])
        # print("First 10 shape values:")
        # print(np.round(np_shape[0][:10], 2))
        # print("First 10 expression values:")
        # print(np.round(np_exp[0][:10], 2))
        return np_verts, np_scale, np_shape, np_exp, np_trans, np_rot, np_pose

In [3]:
def run_2d_lmk_fitting(model_fname, flame_lmk_path, target_img_path, target_lmk_path, out_path):
    if not os.path.exists(out_path):
        os.makedirs(out_path)

    lmk_face_idx, lmk_b_coords = load_embedding(flame_lmk_path)
    target_img = cv2.imread(target_img_path)
    lmk_2d = np.load(target_lmk_path)

    weights = {}
    # Weight of the landmark distance term
    weights['lmk'] = 1.0
    # Weight of the shape regularizer
    weights['shape'] = 1e-3
    # Weight of the expression regularizer
    weights['expr'] = 1e-3
    # Weight of the neck pose (i.e. neck rotationh around the neck) regularizer
    weights['neck_pose'] = 100.0
    # Weight of the jaw pose (i.e. jaw rotation for opening the mouth) regularizer
    weights['jaw_pose'] = 1e-3
    # Weight of the eyeball pose (i.e. eyeball rotations) regularizer
    weights['eyeballs_pose'] = 10.0

    # result_mesh, result_scale = fit_lmk2d(target_img, lmk_2d, model_fname, lmk_face_idx, lmk_b_coords, weights)
    np_verts, np_scale, np_shape, np_exp, np_trans, np_rot, np_pose = fit_lmk2d(target_img, lmk_2d, model_fname, lmk_face_idx, lmk_b_coords, weights)
    fname = os.path.join(out_path, os.path.splitext(os.path.basename(target_img_path))[0])
    np.save(fname + "_" + "np_verts", np_verts)
    np.save(fname + "_" + "np_scale", np_scale)
    np.save(fname + "_" + "np_shape", np_shape)
    np.save(fname + "_" + "np_exp", np_exp)
    np.save(fname + "_" + "np_trans", np_trans)
    np.save(fname + "_" + "np_rot", np_rot)
    np.save(fname + "_" + "np_pose", np_pose)

In [4]:
model_fname = "./generic_model.pkl"
flame_lmk_path = "./flame_static_embedding.pkl"
out_path = "./results"

In [6]:
for i in range(75, 76):
    print()
    print()
    print()
    print(i)
    file_name = f"frame_{str(i).zfill(3)}"
    target_img_path = "results/" + file_name + ".jpg"
    target_lmk_path = "results/" + file_name + "_lmks.npy"
    run_2d_lmk_fitting(model_fname, flame_lmk_path, target_img_path, target_lmk_path, out_path)




75
Optimize rigid transformation
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 0.218029
  Number of iterations: 23
  Number of functions evaluations: 26
Optimize model parameters
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 0.024641
  Number of iterations: 310
  Number of functions evaluations: 339
Fitting done


In [29]:
target_img_path = "./imgHQ00039.jpeg"
target_lmk_path = "./imgHQ00039_lmks.npy"
run_2d_lmk_fitting(model_fname, flame_lmk_path, target_img_path, target_lmk_path, out_path)

Instructions for updating:
Use Variable.read_value. Variables in 2.X are initialized automatically both in eager and graph (inside tf.defun) contexts.
Optimize rigid transformation
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 0.175327
  Number of iterations: 23
  Number of functions evaluations: 27
Optimize model parameters
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 0.037823
  Number of iterations: 182
  Number of functions evaluations: 196
Fitting done
First 10 shape values:
[ 0.71 -0.34 -0.7   0.07 -0.45 -0.05  0.39 -0.18 -0.03 -0.45]
First 10 expression values:
[ 2.78  0.15 -0.6   0.06 -0.18  0.64  1.28 -0.03 -0.29 -0.07]


In [30]:
target_img_path = "./imgHQ00088.jpeg"
target_lmk_path = "./imgHQ00088_lmks.npy"
run_2d_lmk_fitting(model_fname, flame_lmk_path, target_img_path, target_lmk_path, out_path)

Optimize rigid transformation
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 0.071781
  Number of iterations: 22
  Number of functions evaluations: 25
Optimize model parameters
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 0.017228
  Number of iterations: 232
  Number of functions evaluations: 256
Fitting done
First 10 shape values:
[ 0.98  0.25  0.47 -0.21  0.37  0.1  -0.17  0.18 -0.1  -0.02]
First 10 expression values:
[ 0.22  1.01  0.12  0.69  0.16  0.09  0.32 -0.36 -0.21  0.46]


In [31]:
target_img_path = "./imgHQ00095.jpeg"
target_lmk_path = "./imgHQ00095_lmks.npy"
run_2d_lmk_fitting(model_fname, flame_lmk_path, target_img_path, target_lmk_path, out_path)

Optimize rigid transformation
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 0.089902
  Number of iterations: 34
  Number of functions evaluations: 42
Optimize model parameters
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 0.027501
  Number of iterations: 284
  Number of functions evaluations: 310
Fitting done
First 10 shape values:
[-0.1  -0.07 -0.47 -0.25  0.09 -0.45  0.62  0.03 -0.38  0.03]
First 10 expression values:
[ 2.09 -0.29 -0.44 -0.07 -0.13  0.22  0.34 -0.11  0.04 -0.04]


In [32]:
target_img_path = "./imgHQ01148.jpeg"
target_lmk_path = "./imgHQ01148_lmks.npy"
run_2d_lmk_fitting(model_fname, flame_lmk_path, target_img_path, target_lmk_path, out_path)

Optimize rigid transformation
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 0.021805
  Number of iterations: 9
  Number of functions evaluations: 13
Optimize model parameters
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 0.012396
  Number of iterations: 82
  Number of functions evaluations: 87
Fitting done
First 10 shape values:
[ 0.38  0.    0.12  0.05  0.01  0.06 -0.12 -0.11  0.05  0.04]
First 10 expression values:
[ 0.52  0.14  0.06  0.12  0.04  0.2   0.45 -0.09 -0.2   0.26]


In [33]:
target_img_path = "./4.png"
target_lmk_path = "./4_lmks.npy"
run_2d_lmk_fitting(model_fname, flame_lmk_path, target_img_path, target_lmk_path, out_path)

Optimize rigid transformation
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 0.028856
  Number of iterations: 26
  Number of functions evaluations: 30
Optimize model parameters
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 0.015275
  Number of iterations: 277
  Number of functions evaluations: 303
Fitting done
First 10 shape values:
[ 0.75 -0.06 -0.06 -0.2   0.14  0.11 -0.07 -0.18  0.09 -0.04]
First 10 expression values:
[ 0.13  0.4  -0.04  0.22  0.26  0.2  -0.24 -0.04 -0.13  0.25]


In [34]:
target_img_path = "./sad_small.jpg"
target_lmk_path = "./sad_small_lmks.npy"
run_2d_lmk_fitting(model_fname, flame_lmk_path, target_img_path, target_lmk_path, out_path)

Optimize rigid transformation
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 0.098053
  Number of iterations: 21
  Number of functions evaluations: 26
Optimize model parameters
INFO:tensorflow:Optimization terminated with:
  Message: b'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
  Objective function value: 0.031389
  Number of iterations: 410
  Number of functions evaluations: 447
Fitting done
First 10 shape values:
[ 1.5  -0.62 -0.37  0.14 -0.92 -0.17  0.22 -0.58  0.54 -0.2 ]
First 10 expression values:
[-0.23  0.32  0.33 -0.36 -0.19  0.17 -0.25  0.23  0.25 -0.41]
